# Day 1

## Imports and data loading

In [1]:
from utils import get_input, load_data

day = 15


In [2]:
get_input(day)


Data saved


In [2]:
data = load_data(day, list_type="line", number=False)
test_data = [
    "1163751742",
    "1381373672",
    "2136511328",
    "3694931569",
    "7463417111",
    "1319128137",
    "1359912421",
    "3125421639",
    "1293138521",
    "2311944581",
]
test_answer_1 = 40
test_answer_2 = None


## Part one

In [35]:
# Isn't this just djikstra, but with costs on nodes instead of edges?
# Hmm, so I could make a directed graph where edge cost = cost of end node.
# Though not sure dijkstra works with directed graphs. But something will.
import networkx as nx


class Cave:
    def __init__(self, input_rows):
        """Turn the input into a directed graph where edge weight is the weight of 
        the node at the end of that path."""
        self.rows = [[int(i) for i in row] for row in input_rows]
        self.width = len(self.rows[0])
        self.height = len(self.rows)
        self.graph = nx.DiGraph()
        # Make the grid
        for j, y in enumerate(self.rows):
            for i, x in enumerate(y):
                self.graph.add_node((j, i), cost=x)

        # Add the edges to the grid
        for node in self.graph.nodes:
            if node[0] > 0:
                end = (node[0] - 1, node[1])
                self.graph.add_edge(node, end, cost=self.graph.nodes[end]["cost"])
                self.graph.add_edge(end, node, cost=self.graph.nodes[node]["cost"])
            if node[0] < self.height - 1:
                end = (node[0] + 1, node[1])
                self.graph.add_edge(node, end, cost=self.graph.nodes[end]["cost"])
                self.graph.add_edge(end, node, cost=self.graph.nodes[node]["cost"])
            if node[1] > 0:
                end = (node[0], node[1] - 1)
                self.graph.add_edge(node, end, cost=self.graph.nodes[end]["cost"])
                self.graph.add_edge(end, node, cost=self.graph.nodes[node]["cost"])
            if node[1] < self.width - 1:
                end = (node[0], node[1] + 1)
                self.graph.add_edge(node, end, cost=self.graph.nodes[end]["cost"])
                self.graph.add_edge(end, node, cost=self.graph.nodes[node]["cost"])

    def __repr__(self):
        """Draw the graph, including edge weights and node weights."""
        costs = nx.get_node_attributes(self.graph, "cost")
        edge_costs = nx.get_edge_attributes(self.graph, "cost")
        pos = nx.spring_layout(self.graph)
        nx.draw(self.graph, pos, with_labels=True, labels=costs)
        nx.draw_networkx_edge_labels(self.graph, pos, edge_labels=edge_costs)
        return ""

    def get_shortest_path(self):
        """Run djikstra to find the shortest path."""
        return nx.shortest_path(
            self.graph,
            source=(0, 0),
            target=(self.height - 1, self.width - 1),
            weight="cost",
            method="dijkstra",
        )

    def measure_shortest_path(self):
        """Add up the node costs in the shortest path and subtract the start node."""
        path = self.get_shortest_path()
        start = self.graph.nodes[(0, 0)]["cost"]
        return sum([self.graph.nodes[node]["cost"] for node in path]) - start


In [37]:
test_cave = Cave(test_data)
assert test_cave.measure_shortest_path() == test_answer_1

cave = Cave(data)
cave.measure_shortest_path()


435

## Part two